In [2]:
%matplotlib inline
import os
import sys
import numpy as np
import pandas as pd
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import linear_model as lm
from sklearn.metrics import explained_variance_score, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import itertools
cwd = os.getcwd()
# Add directories for modules and data files
dirs = [x[0] for x in os.walk(cwd)]
[sys.path.append(d) for d in dirs]
data_dir = cwd+'\\Input\\DataFiles\\' # Manual way; TODO: find a relative way to do this automatically
vegas_dir = cwd+'\\Input\\VegasOdds\\'
elo_out_dir = cwd+'\\Output\\TestElos\\'
import elo

In [4]:
### Load Regular Season Results
# Start at 2003 because advanced stats can only be calculated that far back

start = 2003
rs = pd.read_csv(data_dir+'RegularSeasonDetailedResults.csv')
rs = rs.loc[rs_c.Season>=start]
rs = rs.drop(['NumOT'], axis=1)
rs = rs.reset_index(drop=True)
rs.tail()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,WFGM,WFGA,WFGM3,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
82036,2018,132,1153,56,1222,55,N,20,46,4,...,23,10,12,9,20,12,12,8,2,19
82037,2018,132,1172,58,1348,57,N,19,50,6,...,23,9,17,11,26,12,14,3,4,16
82038,2018,132,1209,74,1426,61,N,25,56,8,...,25,23,28,19,24,9,13,1,5,16
82039,2018,132,1246,77,1397,72,N,25,50,7,...,27,16,20,18,16,14,10,4,3,21
82040,2018,132,1335,68,1217,65,N,24,52,8,...,18,12,12,5,29,6,8,2,2,17


In [9]:
elo_train = pd.read_csv(elo_out_dir+"EloTrainK65HCA130.csv", index_col=0)
elo_train.tail()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WMargin,WSeasonElo,LSeasonElo,SeasonEloDiff
2089,2018,146,1437,71,1403,59,-12,2188.106007,1973.857023,-214.248983
2090,2018,146,1242,85,1181,81,-4,2084.816709,2082.803818,-2.012892
2091,2018,152,1276,69,1260,57,-12,2153.645824,1890.909689,-262.736135
2092,2018,152,1437,95,1242,79,-16,2188.106007,2084.816709,-103.289297
2093,2018,154,1437,79,1276,62,-17,2188.106007,2153.645824,-34.460183


### Calculating Advanced Season Stats

In [ ]:
def get_sum(df, stat, w_mask, l_mask):
    summed = df[w_mask]['W'+stat].sum() + df[l_mask]['L'+stat].sum()
    return summed

def get_sum_opp(df, stat, w_mask, l_mask):
    summed = df[l_mask]['W'+stat].sum() + df[w_mask]['L'+stat].sum()
    return summed

def percent(a, b):
    return a/(a + b)

def rate(a, b):
    return a / b

## Strength of Schedule

In [ ]:

import itertools

def get_schedule(teamID, df_season, teamID_cols=['WTeamID', 'LTeamID']):
    
    teams = (df_season.loc[(df_season.WTeamID==teamID)|(df_season.LTeamID==teamID), teamID_cols]).values
    # flatten list
    teams = list(itertools.chain(*teams))
    
    # remove the current team's ID from list of teams
    teams = list(filter(lambda team: team != teamID, teams))

    return teams

# Build dictionary of season schedules
schedules = dict()
for season in tourney_teams_by_season.keys():
    season_schedule = dict()
    for team in tourney_teams_by_season[season]:
        df_season = rs_c.loc[(rs_c.Season==season)]
        team_schedule = get_schedule(team, df_season)
        season_schedule[team] = team_schedule
    schedules[season] = season_schedule
    
# Calculate Average Opponent Elo Ratings
df_stats['AvgOppElo'] = None

for season in schedules.keys():
    for team in schedules[season].keys():
        opps = schedules[season][team]
        opp_elos = [elos.loc[(elos.Season==season) & (elos.TeamID==opp), 'SeasonElo'].values[0] for opp in opps]
        avg_opp_elo = np.mean(opp_elos)
        df_stats.loc[(df_stats.Season==season) & (df_stats.TeamID==team), 'AvgOppElo'] = avg_opp_elo
        
df_stats.tail(16)

## Momentum

### Last n elos

In [ ]:
def last_n_elos(elo_history, season, team, elo_col = 'SeasonElo', n=10):
    '''
    Takes input:
    df_elos: a chronologically sorted dataframe of regular season elo ratings
    team: team ID of elo ratings of interest
    season: season of elo ratings of interest
    n: the last n elo ratings from the final game
    '''
    row = [season, team]
    last_n_elos = elo_history.loc[(elo_history.Season == season) & (elo_history.TeamID == team), elo_col].tail(n).values
    row += list(last_n_elos)
    return row

In [ ]:
n=12
df_last_n_elos = df_stats.copy()
to_drop = ['Seed','AvgOppElo']
df_last_n_elos.drop(to_drop, axis=1, inplace=True)
last_n_basic_cols = df_last_n_elos.columns.values
elo_cols = ['FinalElo{}'.format(i) for i in range(n,0,-1)]
df_last_n_elos = pd.concat([df_last_n_elos,pd.DataFrame(columns=elo_cols)])
df_last_n_elos = df_last_n_elos[list(last_n_basic_cols)+list(elo_cols)]
print("Assigning last n elos...")
# Index values: 0 = Season, 1 = TeamID
#df_last_n_elos[elo_cols] = df_last_n_elos.apply(lambda x: last_n_elos(elo_history, x[0], x[1], n=n), axis=1)
df_last_n_elos = df_last_n_elos.apply(lambda x: last_n_elos(elo_history, x[0], x[1], n=n), axis=1)
df_last_n_elos.tail(16)